

## BERT

In [1]:
# import transformers
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

D:\资料\学习产生的资料\数据科学导引\DataScience\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'repo'

In [5]:
train = train_data.copy(deep = True)
test = test_data.copy(deep = True)

(156060, 4) (66292, 3)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [7]:
train['Phrase_Length'] = train['Phrase'].apply(len)

# 获取短语长度的统计信息
length_stats = train['Phrase_Length'].describe()
print(length_stats)

# 如果想查看短语长度的范围（最小值和最大值）
min_length = train['Phrase_Length'].min()
max_length = train['Phrase_Length'].max()

print(f"最小长度: {min_length}")
print(f"最大长度: {max_length}")


count    156060.000000
mean         40.217224
std          38.154130
min           1.000000
25%          14.000000
50%          26.000000
75%          53.000000
max         283.000000
Name: Phrase_Length, dtype: float64
最小长度: 1
最大长度: 283


In [9]:
# 加载 BERT 分词器
tokenizer = AutoTokenizer.from_pretrained("/lustre/user/liclab/lisky/buyf/Class/Introduce2Data/bert_base_uncased")

# 分词并编码
encoded_inputs = tokenizer(
    train["Phrase"].tolist(),  # 输入文本列表
    padding=True,            # 自动填充到相同长度
    truncation=True,         # 截断超长序列（BERT 最大长度 512）
    max_length=128,          # 最大序列长度（可以调整）
    return_tensors="pt"      # 返回 PyTorch 张量
)

print(encoded_inputs.keys())  # 包括 input_ids, attention_mask


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [10]:
# 打印 token_ids, token_type_ids, attention_mask
print("Input IDs:", encoded_inputs['input_ids'][1])
print("Token Type IDs:", encoded_inputs['token_type_ids'][1])
print("Attention Mask:", encoded_inputs['attention_mask'][1])

Input IDs: tensor([  101,  1037,  2186,  1997,  9686, 17695, 18673, 14313,  1996, 15262,
         3351,  2008,  2054,  2003,  2204,  2005,  1996, 13020,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Token Type IDs: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [11]:

# 指定本地模型路径
local_model_path = "/lustre/user/liclab/lisky/buyf/Class/Introduce2Data/bert_base_uncased"

# 加载模型
model = AutoModelForSequenceClassification.from_pretrained(
    local_model_path,  
    num_labels=5       
)

# 查看模型结构
print(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /lustre/user/liclab/lisky/buyf/Class/Introduce2Data/bert_base_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.3, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
from transformers import Trainer, TrainingArguments,EarlyStoppingCallback,EvalPrediction

In [13]:
# 训练参数
batch_size = 64  # 根据你的机器配置调整
metric_name = 'f1'  # 假设你选择F1分数作为最佳模型的评估指标

args = TrainingArguments(
    output_dir="./results",  # 模型保存路径
    eval_strategy="epoch",   # 每个 epoch 后评估
    save_strategy="epoch",   # 每个 epoch 后保存模型
    learning_rate=1e-5,      # 降低学习率
    per_device_train_batch_size=batch_size,  # 训练时的批大小
    per_device_eval_batch_size=batch_size,   # 评估时的批大小
    num_train_epochs= 50,     # 训练的总轮数
    weight_decay=0.1,       # 权重衰减（L2 正则化）
    load_best_model_at_end=True,  # 加载最优模型
    metric_for_best_model=metric_name,  # 选择最佳模型的指标
    logging_dir='./logs',   # 日志文件保存路径
    logging_steps=10,       # 每 10 步记录一次日志
    eval_steps=500,         # 每 500 步进行一次评估
    warmup_steps=500,       # 学习率热启动步数
    fp16=True,              # 启用混合精度训练
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

def multi_class_metrics(predictions, labels):
    softmax = torch.nn.Softmax(dim=-1) 
    probs = softmax(torch.Tensor(predictions))  # 获取每个类别的概率

    # 使用 argmax 将预测值转换为类别标签
    y_pred = np.argmax(probs, axis=1)

    # one-hot 编码
    y_true = labels

    # 计算 F1 score
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    
    # 计算 ROC AUC（需要每个类别的概率）
    roc_auc = roc_auc_score(y_true, probs, multi_class='ovr', average='macro')

    # 计算准确率
    accuracy = accuracy_score(y_true, y_pred)

    # 返回所有指标
    metrics = {
        'f1': f1_micro_average,
        'roc_auc': roc_auc,
        'accuracy': accuracy
    }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_class_metrics(predictions=preds, labels=p.label_ids)
    return result



In [15]:
labels = train['Sentiment'].tolist()  # 情感标签列表
# 将标签转换为张量
labels_tensor = torch.tensor(labels, dtype=torch.long)  # 确保类型为 torch.long

In [16]:
print(labels_tensor)

tensor([1, 2, 2,  ..., 3, 2, 2])


In [18]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,Dataset,DataLoader

# 假设 encoded_inputs_dict 中包含 'input_ids', 'attention_mask', 'token_type_ids' 和 labels_tensor
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']
token_type_ids = encoded_inputs['token_type_ids']
labels = labels_tensor

# 使用 train_test_split 将数据集分割为训练集和验证集（80% 训练，20% 验证）
train_inputs, val_inputs, train_attention_mask, val_attention_mask, train_token_type_ids, val_token_type_ids, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, token_type_ids, labels, test_size=0.2, random_state=42
)

# 自定义 Dataset 类
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, token_type_ids, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.labels = labels
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        # 返回一个字典
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'token_type_ids': self.token_type_ids[idx],
            'labels': self.labels[idx]
        }

# 创建训练集和验证集的 Dataset 实例
train_dataset = CustomDataset(train_inputs, train_attention_mask, train_token_type_ids, train_labels)
val_dataset = CustomDataset(val_inputs, val_attention_mask, val_token_type_ids, val_labels)

# 查看数据格式
print(train_dataset[0])  # 应该是字典形式的输出

# 输出数据集大小
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")


{'input_ids': tensor([  101, 18178,  2229,  2100,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [19]:
# 配置早停回调
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)  # 在 2 次验证损失不下降时停止训练

# 设置 Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,  # 训练数据集
    eval_dataset=val_dataset,  # 验证数据集
    tokenizer=tokenizer,  # 使用的分词器
    compute_metrics=compute_metrics,  # 计算评估指标
    callbacks=[early_stopping]  # 添加早停回调
)


/tmp/ipykernel_3901173/1360408393.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
# 训练模型
trainer.train()

/lustre/user/liclab/lisky/buyf/ENTER/envs/d2l/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.947900,0.877695,0.639017,0.873499,0.639017
2,0.884500,0.830779,0.645969,0.892717,0.645969
3,0.804300,0.804763,0.659650,0.900052,0.659650
4,0.802800,0.825921,0.647571,0.900903,0.647571
5,0.747500,0.822857,0.650583,0.903984,0.650583


/lustre/user/liclab/lisky/buyf/ENTER/envs/d2l/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/lustre/user/liclab/lisky/buyf/ENTER/envs/d2l/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/lustre/user/liclab/lisky/buyf/ENTER/envs/d2l/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/lustre/user/liclab/lisky/buyf/ENTER/envs/d2l/lib/python3.9/site-packages/to

TrainOutput(global_step=2440, training_loss=0.9067691251879832, metrics={'train_runtime': 1502.8815, 'train_samples_per_second': 4153.621, 'train_steps_per_second': 16.235, 'total_flos': 2.56638858205824e+16, 'train_loss': 0.9067691251879832, 'epoch': 5.0})

In [24]:
# 训练结束后，最优模型已经被加载到 model 中
best_model = trainer.model  # 获取最优模型

model_save_path = "/lustre/user/liclab/lisky/buyf/Class/Introduce2Data/bast_model.pt"  # 设置保存路径
torch.save(best_model, model_save_path)

In [25]:
test["Phrase"] = test["Phrase"].fillna("")  # 用空字符串替代 NaN

# 对测试集的每一行进行分词和编码
encoded_inputs_test = tokenizer(
    test["Phrase"].tolist(),  # 测试集的文本列表
    padding=True,                  # 填充为相同长度
    truncation=True,               # 截断超长序列
    max_length=128,                # 最大序列长度
    return_tensors="pt"            # 返回 PyTorch 张量
)

# 输出看看处理结果
print(encoded_inputs_test.keys())  # 应该返回 input_ids 和 attention_mask


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [29]:
best_model_cpu = trainer.model.to('cpu')

# 
best_model_cpu.eval()
# 使用模型进行推理
with torch.no_grad():
    outputs = best_model_cpu(**encoded_inputs_test)

# 获取预测结果
predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

In [32]:
# 将预测结果与 PhraseId 合并
test_results = pd.DataFrame({
    "PhraseId": test["PhraseId"],  # 从测试集读取 PhraseId
    "Sentiment": predictions       # 模型预测的情感标签
})

# 保存为 CSV 文件
output_path = "/lustre/user/liclab/lisky/buyf/Class/Introduce2Data/test_results.csv"  # 指定保存路径
test_results.to_csv(output_path, index=False)

print(f"预测结果已保存至 {output_path}")

预测结果已保存至 /lustre/user/liclab/lisky/buyf/Class/Introduce2Data/test_results.csv
